In [17]:
from dotenv import load_dotenv      # loads environment variables from a .env file into Python environment
import os                           # access environment variables once they've been loaded into the Python environmen
from langchain_groq import ChatGroq   # Access Groq chat models

In [18]:
print(os.getenv('GROQ_API_KEY'))

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


In [19]:
from langchain_core.prompts import ChatPromptTemplate             

#  Creating a template for the prompt
template = """
Think You are data analyst in MySQL. Based on the schema below wirte MySql queries to answer the following question:
Generate an SQL query for the following request:\n{question}\nOnly return the SQL query without any explanations or formatting:
Also when providing the sql query remove \n and \ from th content:
Example question How many customer are there:
then out put should be as SELECT COUNT(*) FROM customers:
Your Turn:
{schema}   

Question: {question}

Sql Query:
"""

prompt = ChatPromptTemplate.from_template(template)

In [20]:
# prompt.format(schema= "myshema", question="my question")

In [21]:
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase

# Define the connection URI
mysql_uri = "mysql+mysqlconnector://root:@localhost:3306/classicmodels"

# Create an SQLAlchemy engine
engine = create_engine(mysql_uri)

# Pass the engine to SQLDatabase
db = SQLDatabase(engine)  # db is an instance of SQLDatabase


In [22]:
# db.run("SQL Query") # runs the SQL query and returns the result
db.run("show tables") # returns a list of tables in the database

"[('customers',), ('employees',), ('offices',), ('orderdetails',), ('orders',), ('payments',), ('productlines',), ('products',)]"

In [23]:
# Get the schema of the database
# Schema is a list of dictionaries, where each dictionary represents a column in a table

def get_schema(_):    # For the RunnablePassthrough towork function should be consist with at least one parameter
    schema = db.get_table_info()
    return schema

In [24]:
# get_schema(None)

In [31]:
from langchain_core.output_parsers import StrOutputParser  # Get the Out Put as a String
from langchain_core.runnables import RunnablePassthrough   
from langchain_openai import ChatOpenAI                    # Importing the OpenAI Chat Model

# llm = ChatOpenAI(
#     temperature=0.5,
#     max_tokens=100,
#     model = "gpt-3.5-turbo"

# )

llm = ChatGroq(model="llama3-8b-8192",temperature = 0.5)

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema) 
    | prompt
    | llm
    | StrOutputParser()
)


In [32]:
user_question = 'How many customer are there'
sql_chain.invoke({"question": user_question})

'SELECT COUNT(*) FROM customers'

In [33]:
template = """Think you are Data Analyst and use Mysql databases. Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)


In [34]:
def run_query(query):
    return db.run(query)


In [35]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema = get_schema,
        response = lambda vars: run_query(vars["query"])
    )
    | prompt_response
    | llm
    | StrOutputParser()
)


In [36]:
user_question = 'Give me the customer name, phone and payment amount greater than 100000?'
full_chain.invoke({"question": user_question})

'Based on the provided table schema and data, here is the natural language response:\n\nThe customers who have made payments greater than $100,000 are:\n\n* Mini Gifts Distributors Ltd., with phone number 4155551450, has made payments of $101,244.59 and $111,654.40.\n* Euro+ Shopping Channel, with phone number (91) 555 94 44, has made payments of $116,208.40 and $120,166.58.\n* Dragon Souveniers, Ltd., with phone number +65 221 7555, has made a payment of $105,743.00.\n\nThese customers have made significant payments, indicating a high level of spending or investment.'